# Обучение модели Word2Vec на текстовых данных социальной сети VK, содержащих эмодзи

### Импорт необходимых библиотек

In [1]:
!pip install --proxy 10.0.0.1:3128 gensim pandas nltk pymorphy2 emoji

Using legacy 'setup.py install' for emoji, since package 'wheel' is not installed.
Using legacy 'setup.py install' for docopt, since package 'wheel' is not installed.
    Running setup.py install for docopt: started
    Running setup.py install for docopt: finished with status 'done'
    Running setup.py install for emoji: started
    Running setup.py install for emoji: finished with status 'done'


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import numpy as np

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [6]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_ru.zip.


True

In [7]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

### Подготовка датасета со смайликами

##### Чтение датасета 

In [40]:
emojis = pd.read_csv('datasets/marking.csv')

In [41]:
emojis = emojis.drop(columns=['image_id', 'name', 'group', 'sub_group', 'train'])

In [42]:
emojis

,text,emoji
0,Ухмыляющееся лицо,😀
1,Ухмыляющееся лицо с большими глазами,😃
2,Улыбающееся лицо с улыбающимися глазами,😄
3,Сияющее лицо с улыбающимися глазами,😁
4,Ухмыляющееся щурящееся лицо,😆
...,...,...
2744,Флаг: Замбия,🇲🇿
2745,Флаг: Зимбабве,🇿🇼
2746,Флаг: Англия,🏴󠁧󠁢󠁥󠁮󠁧󠁿
2747,Флаг: Шотландия,🏴󠁧󠁢󠁳󠁣󠁴󠁿


##### Токенизация описания эмодзи

In [43]:
regular = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
full_tokens = []
for text in emojis['text']:
  text = re.sub(regular, '', text)
  tokens = word_tokenize(text, language='russian')
  full_tokens.append(tokens)
emojis['tokens'] = full_tokens

In [44]:
emojis= emojis.drop('text', axis = 1)

In [45]:
emojis

,emoji,tokens
0,😀,"[Ухмыляющееся, лицо]"
1,😃,"[Ухмыляющееся, лицо, с, большими, глазами]"
2,😄,"[Улыбающееся, лицо, с, улыбающимися, глазами]"
3,😁,"[Сияющее, лицо, с, улыбающимися, глазами]"
4,😆,"[Ухмыляющееся, щурящееся, лицо]"
...,...,...
2744,🇲🇿,"[Флаг, Замбия]"
2745,🇿🇼,"[Флаг, Зимбабве]"
2746,🏴󠁧󠁢󠁥󠁮󠁧󠁿,"[Флаг, Англия]"
2747,🏴󠁧󠁢󠁳󠁣󠁴󠁿,"[Флаг, Шотландия]"


##### Приведение к нормальной форме всех слов в описании

In [46]:
filtered = []
morph = pymorphy2.MorphAnalyzer()
for tokens in emojis['tokens']:
  filtered_tokens = []
  for token in tokens:
    token = morph.normal_forms(token)[0]
    if token not in stopwords.words('russian'):
      filtered_tokens.append(token)
  filtered.append(filtered_tokens)

In [47]:
emojis['filtered'] = filtered

In [48]:
emojis = emojis.drop('tokens', axis = 1)

In [49]:
emojis

,emoji,filtered
0,😀,"[ухмыляться, лицо]"
1,😃,"[ухмыляться, лицо, больший, глаз]"
2,😄,"[улыбаться, лицо, улыбаться, глаз]"
3,😁,"[сиять, лицо, улыбаться, глаз]"
4,😆,"[ухмыляться, щуриться, лицо]"
...,...,...
2744,🇲🇿,"[флаг, замбия]"
2745,🇿🇼,"[флаг, зимбабве]"
2746,🏴󠁧󠁢󠁥󠁮󠁧󠁿,"[флаг, англия]"
2747,🏴󠁧󠁢󠁳󠁣󠁴󠁿,"[флаг, шотландия]"


##### Объединение смайлика с описанием для получения единого предложения для дальнейшего использования датасета для обучения модели

In [50]:
for e in emojis['emoji']:
  ind = emojis[emojis['emoji'] == e].index
  ind.to_list()
  emojis.at[ind[0], 'filtered'].append(e)

In [51]:
emojis = emojis.drop('emoji', axis = 1)

In [52]:
emojis

,filtered
0,"[ухмыляться, лицо, 😀]"
1,"[ухмыляться, лицо, больший, глаз, 😃]"
2,"[улыбаться, лицо, улыбаться, глаз, 😄]"
3,"[сиять, лицо, улыбаться, глаз, 😁]"
4,"[ухмыляться, щуриться, лицо, 😆]"
...,...
2744,"[флаг, замбия]"
2745,"[флаг, зимбабве, 🇿🇼]"
2746,"[флаг, англия, 🏴󠁧󠁢󠁥󠁮󠁧󠁿]"
2747,"[флаг, шотландия, 🏴󠁧󠁢󠁳󠁣󠁴󠁿]"


In [55]:
type(emojis['filtered'].iloc[1])

list

In [56]:
emojis_data = emojis['filtered'].values.tolist()

In [57]:
emojis_data[:5]

[['ухмыляться', 'лицо', '😀'],
 ['ухмыляться', 'лицо', 'больший', 'глаз', '😃'],
 ['улыбаться', 'лицо', 'улыбаться', 'глаз', '😄'],
 ['сиять', 'лицо', 'улыбаться', 'глаз', '😁'],
 ['ухмыляться', 'щуриться', 'лицо', '😆']]

In [58]:
type(emojis_data)

list

In [59]:
type(emojis_data[1])

list

### Подготовка датасета с комментариями ВК

##### Чтение датасета

In [19]:
colnames = ['index', 'comment']

In [21]:
comments = pd.read_csv('datasets/dataset_vk.csv', names=colnames, header=0, sep='\t')

In [22]:
comments

,index,comment
0,0,['норма']
1,1,['😁']
2,2,"['воронка', 'цветок', 'вода', '💮']"
3,3,"['сша', 'фашизм']"
4,4,"['злодей', 'целый', 'месяц', 'ждать']"
...,...,...
20575,20575,"['с', 'фамилия', '🙄']"
20576,20576,"['наш', 'день', 'повезти', 'мужик', 'фамилия',..."
20577,20577,"['чудо', 'чудной', 'диво', 'дивный']"
20578,20578,"['пригреться', 'уснуть', 'свариться', 'лягушка..."


In [23]:
comments = comments.drop(columns = ['index'],axis = 1)

In [24]:
comments

,comment
0,['норма']
1,['😁']
2,"['воронка', 'цветок', 'вода', '💮']"
3,"['сша', 'фашизм']"
4,"['злодей', 'целый', 'месяц', 'ждать']"
...,...
20575,"['с', 'фамилия', '🙄']"
20576,"['наш', 'день', 'повезти', 'мужик', 'фамилия',..."
20577,"['чудо', 'чудной', 'диво', 'дивный']"
20578,"['пригреться', 'уснуть', 'свариться', 'лягушка..."


In [25]:
vk_df = pd.DataFrame()

In [26]:
vk_df['text'] = comments['comment'].apply(str_to_list)

In [27]:
vk_data = vk_df['text'].values.tolist()

In [28]:
vk_data[:5]

[['норма'],
 ['😁'],
 ['воронка', 'цветок', 'вода', '💮'],
 ['сша', 'фашизм'],
 ['злодей', 'целый', 'месяц', 'ждать']]

In [33]:
type(vk_data)

list

In [34]:
type(vk_data[1])

list

## Загрузка датасета lenta.ru из corus (natasha)

In [9]:
lenta_learn = pd.read_csv('lenta_df_lern.csv')

In [10]:
lenta_learn.head(2)

,Unnamed: 0,text
0,0,"['бой', 'сопоцкина', 'друскеник', 'закончиться..."
1,1,"['министерство', 'народный', 'просвещение', 'в..."


In [14]:
def str_to_list(i):
    new_list = eval(i)
    return new_list

In [15]:
lenta_df = pd.DataFrame()

In [16]:
lenta_df['text'] = lenta_learn['text'].apply(str_to_list)

In [29]:
lenta_data = lenta_df['text'].values.tolist()

In [30]:
lenta_data[:5]

[['бой',
  'сопоцкина',
  'друскеник',
  'закончиться',
  'отступление',
  'германец',
  'неприятель',
  'приблизиться',
  'север',
  'осовца',
  'начать',
  'артиллерийский',
  'борьба',
  'крепость',
  'в',
  'артиллерийский',
  'бой',
  'принимать',
  'участие',
  'тяжёлый',
  'калибр',
  'с',
  'ранний',
  'утро',
  'сентябрь',
  'огонь',
  'достигнуть',
  'значительный',
  'напряжение',
  'попытка',
  'германский',
  'пехота',
  'пробиться',
  'близкий',
  'крепость',
  'отразить',
  'в',
  'галиция',
  'занять',
  'дембица',
  'больший',
  'колонна',
  'отступать',
  'шоссе',
  'перемышль',
  'санок',
  'обстреливаться',
  'высота',
  'наш',
  'батарея',
  'бежать',
  'бросить',
  'парка',
  'обоз',
  'автомобиль',
  'вылазка',
  'гарнизон',
  'перемышль',
  'оставаться',
  'безуспешный',
  'при',
  'продолжаться',
  'отступление',
  'австриец',
  'обнаруживаться',
  'полный',
  'перемешивание',
  'часть',
  'захватываться',
  'новый',
  'партия',
  'пленный',
  'орудие',
  'проч

In [31]:
type(lenta_data)

list

In [32]:
type(lenta_data[0])

list

# Обучение модели w2v

In [35]:
model_init = wv(vector_size=300, min_count=3, window=5)

In [37]:
model_init.build_vocab(emojis_data)

In [38]:
model_init.train(emojis_data, total_examples=model_init.corpus_count, epochs=30)

(3745908, 4474170)

In [39]:
model_init.wv.most_similar('😀')

[('🐽🇺🇦', 0.9206594824790955),
 ('психануть', 0.906537652015686),
 ('ыыы', 0.9044475555419922),
 ('шахта', 0.9039511680603027),
 ('аватарка', 0.9016205668449402),
 ('рашистый', 0.8986945748329163),
 ('трубопровод', 0.8984929323196411),
 ('☺', 0.8953710198402405),
 ('💪', 0.8951533436775208),
 ('нацик', 0.8918470144271851)]